In [ ]:
############################################################
# SageMath Code: Verification of Theorem 3.5
# Comparison between Graph Walk Counts and Theoretical Formulas
############################################################

def verify_theorem_3_5(a_list, b_list):
    """
    1. Builds the tree T(a1...ar; b1...bs).
    2. Computes actual walk counts d0, d1, d2, d3 using matrix powers.
    3. Computes theoretical coefficients from Theorem 3.5, System (1).
    4. Compares them to verify the theorem.
    """
    
    # --- 1. Basic Parameters ---
    r = len(a_list)
    s = len(b_list)
    sigma_a = sum(a_list)
    sigma_b = sum(b_list)
    
    # --- 2. Build Graph (c=0, d=0 for Thm 3.5) ---
    # Using the same building logic as before
    G = Graph(multiedges=False, loops=False)
    u1, u2 = ('u',1), ('u',2)
    G.add_edge(u1, u2)
    
    # Build stars for u1
    for i, ai in enumerate(a_list, start=1):
        w = ('w', i)
        G.add_edge(u1, w)
        for k in range(1, ai+1):
            G.add_edge(w, ('x', i, k))
            
    # Build stars for u2
    for j, bj in enumerate(b_list, start=1):
        z = ('z', j)
        G.add_edge(u2, z)
        for k in range(1, bj+1):
            G.add_edge(z, ('y', j, k))
            
    # --- 3. Compute Actual Walk Vectors ---
    # We need a fixed order to map vertices to matrix indices
    verts = G.vertices(sort=True) # Sort makes it deterministic
    A = G.adjacency_matrix(vertices=verts)
    n = G.order()
    j_vec = vector(ZZ, [1]*n)
    
    # Calculate d0, d1, d2, d3 vectors
    # v_l = A^l * j
    v0 = j_vec
    v1 = A * v0
    v2 = A * v1
    v3 = A * v2
    
    # Helper to get actual d_k for a specific vertex label
    def get_actual_d(v_label):
        idx = verts.index(v_label)
        return (v0[idx], v1[idx], v2[idx], v3[idx])

    # --- 4. Define Theoretical Formulas (from Image Eq 1) ---
    # The system is: c2*d0 + c1*d1 + c0*d2 = d3
    # Matrix columns correspond to coeffs of: [c2, c1, c0] -> [d0, d1, d2]
    # We verify the row: [d0, d1, d2 | d3]
    
    print(f"VERIFICATION REPORT for T({a_list}; {b_list})")
    print(f"Parameters: r={r}, s={s}, sigma_a={sigma_a}, sigma_b={sigma_b}")
    print("="*85)
    print(f"{'Vertex Type':<20} | {'Coeffs (d0, d1, d2) -> RHS (d3)':<35} | {'Check':<10}")
    print("-" * 85)

    augmented_matrix_rows = []

    # --- CHECK 1: Leaves of u1's stars (Leaf of w_i) ---
    # Theory: c2 + c1 + (a_i+1)c0 = a_i + r + 1
    # Vector: [1, 1, a_i+1, a_i+r+1]
    for i, ai in enumerate(a_list, start=1):
        v_label = ('x', i, 1) # Pick the first leaf of i-th star
        act = get_actual_d(v_label)
        
        # Theoretical values from Thm 3.5 Eq (1) line 1
        theo_d0 = 1
        theo_d1 = 1
        theo_d2 = ai + 1
        theo_d3 = ai + r + 1
        theo = (theo_d0, theo_d1, theo_d2, theo_d3)
        
        match = (act == theo)
        status = "[MATCH]" if match else "!! MISMATCH !!"
        print(f"Leaf of w_{i} (a={ai:<2})  | Actual: {act}     | {status}")
        if not match: print(f"                     | Theory: {theo}")
        augmented_matrix_rows.append(list(act))

    # --- CHECK 2: Leaves of u2's stars (Leaf of z_j) ---
    # Theory: c2 + c1 + (b_j+1)c0 = b_j + s + 1
    for j, bj in enumerate(b_list, start=1):
        v_label = ('y', j, 1)
        act = get_actual_d(v_label)
        
        theo = (1, 1, bj + 1, bj + s + 1)
        
        match = (act == theo)
        status = "[MATCH]" if match else "!! MISMATCH !!"
        print(f"Leaf of z_{j} (b={bj:<2})  | Actual: {act}     | {status}")
        augmented_matrix_rows.append(list(act))

    print("-" * 85)

    # --- CHECK 3: Centers of u1's stars (w_i) ---
    # Theory: c2 + (a_i+1)c1 + (a_i+r+1)c0 = a_i^2 + a_i + sigma_a + r + s + 1
    # Note: d1 = a_i+1. d2 = a_i+r+1.
    for i, ai in enumerate(a_list, start=1):
        v_label = ('w', i)
        act = get_actual_d(v_label)
        
        theo_d0 = 1
        theo_d1 = ai + 1
        theo_d2 = ai + r + 1 # d2(w_i) = neighbors(leaves)+neighbor(u1) = a_i*1 + (r+1) = a_i+r+1 ??
                             # WAIT. u1 degree is r+1. Leaves degree is 1.
                             # sum of neighbors deg: a_i*(1) + (r+1) = a_i+r+1. Correct.
        
        theo_d3 = ai**2 + ai + sigma_a + r + s + 1
        theo = (theo_d0, theo_d1, theo_d2, theo_d3)
        
        match = (act == theo)
        status = "[MATCH]" if match else "!! MISMATCH !!"
        print(f"Center w_{i} (a={ai:<2})   | Actual: {act}  | {status}")
        if not match: print(f"                     | Theory: {theo}")
        augmented_matrix_rows.append(list(act))

    # --- CHECK 4: Centers of u2's stars (z_j) ---
    # Theory: c2 + (b_j+1)c1 + (b_j+s+1)c0 = b_j^2 + b_j + sigma_b + r + s + 1
    for j, bj in enumerate(b_list, start=1):
        v_label = ('z', j)
        act = get_actual_d(v_label)
        
        theo = (1, bj + 1, bj + s + 1, bj**2 + bj + sigma_b + r + s + 1)
        
        match = (act == theo)
        status = "[MATCH]" if match else "!! MISMATCH !!"
        print(f"Center z_{j} (b={bj:<2})   | Actual: {act}  | {status}")
        augmented_matrix_rows.append(list(act))

    print("-" * 85)

    # --- CHECK 5: Hub u1 ---
    # Theory: c2 + (r+1)c1 + (sigma_a + r + s + 1)c0 = sigma_a + sigma_b + (r+1)^2 + s
    # Note from Image Eq (1): Coeff of c0 is (sigma_a + r + s + 1). This should be d2(u1).
    v_label = ('u', 1)
    act = get_actual_d(v_label)
    
    theo_d0 = 1
    theo_d1 = r + 1
    theo_d2 = sigma_a + r + s + 1 # Sum of neighbor degrees: sum(a_i+1) + (s+1) = sigma_a+r + s+1. Correct.
    theo_d3 = sigma_a + sigma_b + (r + 1)**2 + s
    theo = (theo_d0, theo_d1, theo_d2, theo_d3)
    
    match = (act == theo)
    status = "[MATCH]" if match else "!! MISMATCH !!"
    print(f"Hub u1             | Actual: {act} | {status}")
    if not match: print(f"                     | Theory: {theo}")
    augmented_matrix_rows.append(list(act))

    # --- CHECK 6: Hub u2 ---
    # Theory: c2 + (s+1)c1 + (sigma_b + r + s + 1)c0 = sigma_a + sigma_b + (s+1)^2 + r
    v_label = ('u', 2)
    act = get_actual_d(v_label)
    
    theo = (1, s + 1, sigma_b + r + s + 1, sigma_a + sigma_b + (s + 1)**2 + r)
    
    match = (act == theo)
    status = "[MATCH]" if match else "!! MISMATCH !!"
    print(f"Hub u2             | Actual: {act} | {status}")
    if not match: print(f"                     | Theory: {theo}")
    augmented_matrix_rows.append(list(act))

    print("="*85)
    print("\nGenerated Augmented Matrix (matches Theorem 3.5 structure):")
    M = Matrix(ZZ, augmented_matrix_rows)
    print(M)
    return M

############################################################
# Run the verification with a specific example
############################################################
# Example: T(2, 4; 3, 2)
# a1=2, a2=4 (r=2)
# b1=3, b2=2 (s=2)
verify_theorem_3_5([2, 4], [3, 2])

Compare computed d_l(v) with system (1) coefficients/RHS
a_list=[2, 4, 1], b_list=[3, 2]

v = ('x', 1, 1)
  computed : d0=1, d1=1, d2=3, d3=6
  system(1): d0=1, d1=1, d2=3, d3=6
  ==> OK
----------------------------------------------------------------------
v = ('x', 2, 1)
  computed : d0=1, d1=1, d2=5, d3=8
  system(1): d0=1, d1=1, d2=5, d3=8
  ==> OK
----------------------------------------------------------------------
v = ('x', 3, 1)
  computed : d0=1, d1=1, d2=2, d3=5
  system(1): d0=1, d1=1, d2=2, d3=5
  ==> OK
----------------------------------------------------------------------
v = ('y', 1, 1)
  computed : d0=1, d1=1, d2=4, d3=6
  system(1): d0=1, d1=1, d2=4, d3=6
  ==> OK
----------------------------------------------------------------------
v = ('y', 2, 1)
  computed : d0=1, d1=1, d2=3, d3=5
  system(1): d0=1, d1=1, d2=3, d3=5
  ==> OK
----------------------------------------------------------------------
v = ('w', 1)
  computed : d0=1, d1=3, d2=6, d3=19
  system(1): d0=1, d

True